In [2]:
import math

def B_uncontrolled_uptake(d,T,N,A,t_u):
    '''
    param d: distance from patient to PoP (m)
    param T: occupancy factor
    param N: number of patients per week
    param A: initial activity (mCi)
    param t_u: uptake time (hr)
    '''

    B_uncontrolled = (5.89 * d**2) / (T * N * A * t_u * 0.83)
    return B_uncontrolled

def B_controlled_uptake(d,T,N,A,t_u):
    '''
    param d: distance from patient to PoP (m)
    param T: occupancy factor
    param N: number of patients per week
    param A: initial activity (mCi)
    param t_u: uptake time (hr)
    '''

    B_controlled = (29.5 * d**2) / (T * N * A * t_u * 0.83)
    return B_controlled

def convert_to_meters(foot,inches):
    m = (foot * 12 * 2.54 + inches * 2.54) * 10e-3
    return m

patients = 30   

https://www.leadshielding.com/products/lead-sheet?srsltid=AfmBOoolOd4EkQaPowMuO1iVG1OUw-Rc67AS5_KHpedf3p6aHrcXgz2m

# Office
Full Occupancy
Uncontrolled

In [3]:
d_office = convert_to_meters(1,0) + 0.3
print(f"distance to office = {d_office} m")
B_uncontrolled_uptake(d_office,1,patients,10,1)

distance to office = 0.6048 m


0.008652458255421687

From Figure 1 in TG 108, t = 32.5 mm of lead = 1.5" lead

# Staff Lounge
T = 1/5 Uncontrolled

In [4]:
d_staff = convert_to_meters(1,0) + 0.3
print(f"distance to staff lounge = {d_staff} # m")
B_uncontrolled_uptake(d_staff, 1/5, patients, 10, 1)

distance to staff lounge = 0.6048 # m


0.043262291277108435

t = 22 m of lead = 1" lead

# Hot Lab
T = 1/4
Controlled room

In [28]:
d_hotlab = convert_to_meters(1,0) + 0.3
print(f"distance to hotlab = {d_hotlab} m")

B_controlled_uptake(d_hotlab,1/2,patients,10,1)

distance to hotlab = 0.6048 m


0.08667148337349398

From Table IV, t = 18 mm of lead = 3/4" lead

# Controlled Corridor
T = 1/5

In [6]:
d_controlled_corridor = convert_to_meters(5,7.5) + 0.3 # m
print(f"distance to controlled corridor = {d_controlled_corridor} m")

B_controlled_uptake(d_controlled_corridor,1/5,patients,10,1)

distance to controlled corridor = 2.0145 m


2.403959887048193

No thickness required but we will use 1/4" in case the patient moves closer to the wall at any point

# Uncontrolled Corridor
T = 1/5

In [7]:
d_uncontrolled_corridor = convert_to_meters(1,0) + 0.3 # m
print(f"distance to uncontrolled corridor = {d_uncontrolled_corridor} m")

B_uncontrolled_uptake(d_uncontrolled_corridor,1/5,patients,10,1)


distance to uncontrolled corridor = 0.6048 m


0.043262291277108435

From Figure 1, t = 22 m of lead = 1" of lead


# Adjacent Uptake Room
Controlled
T = 1/2

In [8]:
d_adjacent_uptake = convert_to_meters(3,9) + 0.3 # m
print(f"distance to adjacent uptake room = {d_adjacent_uptake}")

B_controlled_uptake(d_adjacent_uptake,1/2,patients,10,1)

distance to adjacent uptake room = 1.443


0.4933843012048193

From Figure 1, use t = 6mm of lead

# Ceiling for both uptake rooms
Uncontrolled
T = 1

In [9]:
d_ceiling = convert_to_meters(8,4.31496) + 0.5 # m
print(f"distance to floor above = {d_ceiling} m")
B_uncontrolled_uptake(d_ceiling,1,patients,10,1)

distance to floor above = 3.047999984 m


0.21975859431932915

From Table IV, t = 14 cm of concrete

In [10]:
t_lightweight_ceiling = (14 * 147/115) / 2.54
print(f"{t_lightweight_ceiling} inches of lightweight concrete required")


7.045532351934269 inches of lightweight concrete required


In [11]:
t_imaging = 25 # minutes

R_t_i = 1.443 * (110/t_imaging) * (1-math.exp(-0.693*t_imaging/110))
print(f"R_t_i: {R_t_i}")

F_u = 0.68 # decay factor for F-18 at 1 hour uptake time

def B_uncontrolled_PET(d,T,N,A):
    '''
    param d: distance from source to PoP (m)
    param T: occupancy factor
    N: number of patients/wk
    A: activity in MBq
    '''
    B = 256 * d**2 / (T * N * A * F_u * (t_imaging/60) * R_t_i)
    return B

def B_controlled_PET(d,T,N,A):
    B = 1280 * d**2 / (T * N * A * F_u * (t_imaging/60) * R_t_i)
    return B

activity_mbq = 370 # MBq

K_sec_head = 9e-5 * (20/0.9) * (20.8*380/100)
K_sec_body_1 = 3e-4 * (20/0.9) * (13.8*380/100)
K_sec_body_2 = 3e-4 * (50/1.2) * (13.8*380/100)


# RTP CT 
K_sec_RTP_scans = K_sec_head * 25 + K_sec_body_1 * 40 + K_sec_body_2 * 60
print(f"K_sec_RTP_scans = {K_sec_RTP_scans} mGy/wk")

# CT concurrent with PET
K_sec_transsmission_body = 3e-4 * (100/1.2) * (13.8*200/100) * 50

K_sec_total = K_sec_transsmission_body + K_sec_RTP_scans
print(f"K_sec_total = {K_sec_total} mGy/wk")



R_t_i: 0.925225655037042
K_sec_RTP_scans = 57.266 mGy/wk
K_sec_total = 91.76599999999999 mGy/wk


In [12]:
def find_thickness(B):
    thickness = (1/(2.246*0.547)) * math.log((B**-0.547 + 5.73/0.547) / (1 + 5.73/0.547))
    return thickness

def B_CT(P,T, K_sec,d):
    B = (P/T) / (K_sec/d**2)
    return B

def find_thickness_concrete(B):
    thickness = (1/(0.0383*0.658)) * math.log((B**-0.658 + 0.0142/0.658) / (1 + 0.0142/0.658))
    return thickness

# PET WC Wall

In [13]:
d_pet_wc_wall = convert_to_meters(15,1.26723) + 0.3 # m
print(f"d_pet_wc_wall = {d_pet_wc_wall} m")
pet_wc_B = B_controlled_PET(d_pet_wc_wall, 1/20, patients, activity_mbq)
print(pet_wc_B)

d_pet_wc_wall = 4.904187642 m
211.59519738697628


# CT WC Wall

In [14]:
print(K_sec_total)
test = B_CT(0.1,1/20,K_sec_total,d_pet_wc_wall)
print(test)

print(f"{find_thickness(test)} mm of lead")

91.76599999999999
0.5241822990638259
0.02951759714869584 mm of lead


# Controlled Corridor from PET

In [15]:
d_pet_controlled_corridor = convert_to_meters(16,1.91254) # m
print(d_pet_controlled_corridor)
pet_cc_B = B_controlled_PET(d_pet_controlled_corridor,1/5,patients,activity_mbq)
print(pet_cc_B)

4.925378516
53.356935829058585


# CT Controlled Corridor

In [16]:
print(K_sec_total)
B_CT_controlled_corr = B_CT(0.1,1/5,K_sec_total,d_pet_controlled_corridor)
print(B_CT_controlled_corr)

print(f"{find_thickness(B_CT_controlled_corr)} mm of lead")

91.76599999999999
0.1321805108965977
0.13227879739647863 mm of lead


# PET Utility Room 

In [17]:
d_utilities = convert_to_meters(7,3.63187) + 0.3 # m
print(d_utilities)

B_PET_util = B_uncontrolled_PET(d_utilities,1/40,patients,activity_mbq)
print(B_PET_util)

2.525849498
22.45156295818417


# CT Utility Room

In [29]:
print(K_sec_total)
B_CT_utility_room = B_CT(0.02,1/40,K_sec_total,d_utilities)
print(B_CT_utility_room)

print(f"{find_thickness(B_CT_utility_room)} mm of lead")

91.76599999999999
0.05561899340973216
0.23585347711646804 mm of lead


# PET Control Room

In [19]:
d_pet_control_room = convert_to_meters(23,4.31979) + 0.3 # m
print(d_pet_controlled_corridor)

pet_control_room_B = B_controlled_PET(d_pet_control_room,1,patients,activity_mbq)
print(pet_control_room_B)

4.925378516
24.21942471355457


In [20]:
print(K_sec_total)
B_CT_control_room = B_CT(0.1,1,K_sec_total,d_pet_control_room)
print(B_CT_control_room)

print(f"{find_thickness(B_CT_control_room)} mm of lead")

91.76599999999999
0.05999849658750186
0.22531390339935833 mm of lead


# Ambulance Area

In [21]:
d_ambulance = convert_to_meters(4,2.18021) + 0.3 # m
print(d_ambulance)

B_PET_ambulance = B_uncontrolled_PET(d_ambulance,1/40,patients,activity_mbq)
print(B_PET_ambulance)

1.574577334
8.724883391114826


In [22]:
print(K_sec_total)
B_CT_ambulance = B_CT(0.02,1/40,K_sec_total,d_ambulance)
print(B_CT_ambulance)

print(f"{find_thickness(B_CT_ambulance)} mm of lead")

91.76599999999999
0.021614051223734696
0.39400787253688596 mm of lead


# PET/CT Ceiling

In [23]:
d_ceiling = convert_to_meters(6,17) + 0.5 # m
print(f"d_ceiling = {d_ceiling} m")

B_PET_ceiling = B_uncontrolled_PET(d_ceiling,1,patients,activity_mbq)
print(B_PET_ceiling)

d_ceiling = 2.7606 m
0.6704688668932592


8 cm of standard concrete required

In [24]:
t_lightweight_ceiling_PET = (6 * 147/115) / 2.54
print(f"{t_lightweight_ceiling_PET} inches of lightweight concrete required")


3.019513865114687 inches of lightweight concrete required


In [25]:
print(K_sec_total)
B_CT_ceiling = B_CT(0.02,1,K_sec_total,d_ceiling)
print(B_CT_ceiling)

91.76599999999999
0.0016609446548830726


We use Figure A.3 so 155 mm of standard concrete is required which is 7.8" of lightweight concrete -> NO SHIELDING REQUIRED

# Testing Door of PET control console if it doesn't pass through any shielding

In [26]:
testvalue = convert_to_meters(31,5.88695)
print(testvalue)

testvalue2 = B_controlled_PET(testvalue,1/5,patients,activity_mbq)
print(testvalue2)

9.59832853
202.6295963367611


In [27]:
print(K_sec_total)
testvalue3 = B_CT(0.1,1/5,K_sec_total,testvalue)
print(testvalue3)

91.76599999999999
0.5019719208084256


If we use T=1/5, we need shielding, if we use T=1/8, we still need shielding

Whole control console needs shielding